In [1]:
import numpy as np

from qat.model.component import make_refdict
from qat.model.device import PhysicalBaseband, PhysicalChannel, PulseChannel, Resonator, Qubit
from qat.model.hardware_model import QuantumHardwareModel

In [2]:
def make_Hardware(count=10, connections=3, seed=42):
    rng = np.random.default_rng(seed)
    pick = lambda L, size=3: make_refdict(*rng.choice(L, size=size))
    pick_pulse_channels = lambda pulse_channels, physical_channel: make_refdict(*[pulse_ch for pulse_ch in pulse_channels if pulse_ch.physical_channel == physical_channel])
    physical_basebands = [PhysicalBaseband() for _ in range(count)]
    physical_channels = [PhysicalChannel(baseband=list(pick(physical_basebands, 1).values())[0]) for _ in range(count//5)]
    pulse_channels = [PulseChannel(physical_channel=physical_channel, auxiliary_qubits=[]) for physical_channel in physical_channels]
    resonators = [Resonator(physical_channel=physical_channel, pulse_channels=pick_pulse_channels(pulse_channels=pulse_channels, physical_channel=physical_channels[0])) for physical_channel in list(pick(physical_channels, count//2).values())]
    qubits = [Qubit(physical_channel=physical_channel, pulse_channels=pick_pulse_channels(pulse_channels=pulse_channels, physical_channel=physical_channels[0]), measure_device=list(pick(resonators, 1).values())[0]) for physical_channel in list(pick(physical_channels, count//2).values())] 

    for pulse_channel in pulse_channels:
        pulse_channel.auxiliary_qubits = list(pick(qubits, 3).values())

    return QuantumHardwareModel(physical_basebands=make_refdict(*physical_basebands),
                                physical_channels=make_refdict(*physical_channels),
                                pulse_channels=make_refdict(*pulse_channels),
                                qubits=make_refdict(*qubits),
                                resonators=make_refdict(*resonators))

In [3]:
hw = make_Hardware(count=20)

serialised_hw = hw.model_dump()

deserialised_hw = QuantumHardwareModel(**serialised_hw)

hw._deepequals(deserialised_hw)

True